На лекциях мы обсуждали вероятностный PCA, и что можно получить его с помощью EM алгоритма. Однако есть байесовский способ сделать это с помощью марковских цепей через библиотеку PyMC просто расписав модель и априорные распределения параметров следующим образом, а дальше запустить механизмы сэмплирования:
По аналогии реализуйте Supervised PCA, Partial Least Squares (лекция 3, слайд 23) и CCA.

In [1]:
import pymc as pm
import numpy as np

# Generate synthetic data
np.random.seed(42)
n_samples, n_features, latent_dim = 100, 5, 2
true_W = np.random.randn(n_features, latent_dim)
true_Z = np.random.randn(n_samples, latent_dim)
noise_variance = 0.1
X = true_Z @ true_W.T + np.random.normal(0, np.sqrt(noise_variance), (n_samples, n_features))

# PyMC Model for PPCA
with pm.Model() as ppca_model:
    # Priors
    W = pm.Normal("W", mu=0, sigma=1, shape=(n_features, latent_dim))  # Weight matrix
    Z = pm.Normal("Z", mu=0, sigma=1, shape=(n_samples, latent_dim))  # Latent variables
    sigma = pm.HalfNormal("sigma", sigma=1)  # Noise standard deviation

    # Likelihood
    X_obs = pm.Normal("X_obs", mu=pm.math.dot(Z, W.T), sigma=sigma, observed=X)

    # Inference
    trace = pm.sample(1000, return_inferencedata=True) #количество шагов

# Extract posterior summaries
import arviz as az
summary = az.summary(trace)
print(summary)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [W, Z, sigma]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 8 seconds.
There were 11 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


           mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
W[0, 0]   0.001  0.306  -0.447    0.443      0.138    0.104       6.0   
W[0, 1]  -0.031  0.293  -0.431    0.443      0.131    0.099       6.0   
W[1, 0]   0.743  0.947  -1.164    1.902      0.398    0.298       7.0   
W[1, 1]  -0.048  1.252  -1.756    1.748      0.568    0.429       6.0   
W[2, 0]  -0.132  0.215  -0.399    0.318      0.093    0.069       6.0   
...         ...    ...     ...      ...        ...      ...       ...   
Z[98, 0]  0.275  0.466  -0.681    0.940      0.182    0.135       8.0   
Z[98, 1] -0.043  0.524  -0.838    0.828      0.219    0.163       6.0   
Z[99, 0]  1.359  1.854  -2.160    3.749      0.772    0.576       7.0   
Z[99, 1] -0.011  2.626  -3.748    3.625      1.194    0.901       6.0   
sigma     0.308  0.013   0.285    0.333      0.000    0.000    1781.0   

          ess_tail  r_hat  
W[0, 0]       29.0   1.91  
W[0, 1]       34.0   1.84  
W[1, 0]       19.0   1.55  
W[1, 1]    

In [2]:
# Get latents
Z_mean = trace.posterior["Z"].mean(("chain", "draw"))
print("PCA Scores (Z):", Z_mean)

PCA Scores (Z): <xarray.DataArray 'Z' (Z_dim_0: 100, Z_dim_1: 2)>
array([[-0.19524687,  0.03578691],
       [-0.68284733, -0.01924429],
       [-0.69080091,  0.15894163],
       [-0.08420787,  0.09505976],
       [-0.62327488,  0.12731822],
       [ 0.08571   , -0.13665498],
       [-0.49859769, -0.02349653],
       [-0.03176575,  0.03195127],
       [-0.04846811,  0.09005837],
       [-0.11345605,  0.06812779],
       [ 0.53490297,  0.07163584],
       [-0.28137353,  0.01879957],
       [-0.38928872, -0.09943416],
       [-0.64507785, -0.02327861],
       [-0.13686325,  0.1700104 ],
       [ 0.19883707, -0.06922023],
       [-0.12196608,  0.00830767],
       [-0.4523775 ,  0.14311902],
       [ 0.27109588,  0.03525016],
       [-0.4784607 , -0.03527884],
...
       [-0.43566083,  0.1246813 ],
       [ 0.11598248, -0.0029662 ],
       [ 0.42400571, -0.02080403],
       [ 0.49157972,  0.01076672],
       [ 0.972931  ,  0.03251859],
       [-0.19088329, -0.03987882],
       [ 0.02019678,

In [3]:
# Project new data

W_mean = trace.posterior["W"].mean(("chain", "draw")).to_numpy()

X_new = np.random.randn(10, W_mean.shape[0])  # Shape (10, d), where d = n_features

# Compute Z_new (PCA scores for new data)
W_cov_inv = np.linalg.inv(W_mean.T @ W_mean)  # (k x k)
Z_new = X_new @ W_mean @ W_cov_inv  # Shape (10, k)
Z_new

array([[ -0.29050388,  -5.01373011],
       [ -0.36704571,  -1.91974364],
       [ -2.21321392, -14.26216955],
       [ -0.52869237,  -0.26361886],
       [ -0.15254411,  14.90434332],
       [  0.71032937,  -5.28352923],
       [ -0.27855462,   5.33188441],
       [ -0.56487763,  -1.95835184],
       [ -2.27094648, -17.06994097],
       [  0.17070342,   1.98812989]])

# Supervised PCA

In [4]:
# генерируем синтетические данные для Supervised PCA
np.random.seed(42)
n_samples, n_features, latent_dim = 100, 5, 2
true_W = np.random.randn(n_features, latent_dim)
true_Z = np.random.randn(n_samples, latent_dim)
true_y = true_Z @ np.random.randn(latent_dim, 1) + np.random.normal(0, 0.1, (n_samples, 1))  # целевая переменная
X = true_Z @ true_W.T + np.random.normal(0, 0.1, (n_samples, n_features))  # наблюдаемые данные

# PyMC Model for Supervised PCA
with pm.Model() as spca_model:
    W = pm.Normal("W", mu=0, sigma=1, shape=(n_features, latent_dim))  # матрица весов
    Z = pm.Normal("Z", mu=0, sigma=1, shape=(n_samples, latent_dim))  # скрытые переменные
    sigma = pm.HalfNormal("sigma", sigma=1)  # стандартное отклонение шума
    beta = pm.Normal("beta", mu=0, sigma=1, shape=(latent_dim, 1))  # кфы для целевых переменных
    mu_y = pm.Normal("mu_y", mu=0, sigma=1)  # смещение для целевых переменных

    # ликвидность
    X_obs = pm.Normal("X_obs", mu=pm.math.dot(Z, W.T), sigma=sigma, observed=X)
    y_obs = pm.Normal("y_obs", mu=pm.math.dot(Z, beta) + mu_y, sigma=sigma, observed=true_y)

    # инференс
    trace_spca = pm.sample(1000, return_inferencedata=True)

# Извлечение постериорных суммарных данных
import arviz as az
summary_spca = az.summary(trace_spca)
print(summary_spca)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [W, Z, sigma, beta, mu_y]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 21 seconds.
There were 6 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


             mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
W[0, 0]     0.071  0.214  -0.342    0.471      0.071    0.052       9.0   
W[0, 1]     0.151  0.385  -0.505    0.517      0.181    0.137       6.0   
W[1, 0]     0.050  1.549  -1.913    1.880      0.730    0.554       5.0   
W[1, 1]    -0.205  0.807  -1.946    1.200      0.280    0.205       8.0   
W[2, 0]    -0.027  0.308  -0.372    0.366      0.142    0.108       6.0   
...           ...    ...     ...      ...        ...      ...       ...   
Z[99, 1]   -0.645  1.900  -3.798    3.153      0.688    0.507       7.0   
beta[0, 0]  0.049  1.202  -1.470    1.450      0.566    0.429       6.0   
beta[1, 0] -0.136  0.612  -1.477    0.925      0.212    0.156       8.0   
mu_y       -0.002  0.012  -0.025    0.021      0.000    0.000    4012.0   
sigma       0.100  0.004   0.093    0.106      0.000    0.000    2166.0   

            ess_tail  r_hat  
W[0, 0]         43.0   1.35  
W[0, 1]         28.0   1.96  
W[1, 0]  

In [5]:
# Получаем скрытые переменные (PCA Scores)
Z_mean_spca = trace_spca.posterior["Z"].mean(("chain", "draw"))
print("Supervised PCA Scores (Z):", Z_mean_spca)

Supervised PCA Scores (Z): <xarray.DataArray 'Z' (Z_dim_0: 100, Z_dim_1: 2)>
array([[-7.62715747e-02, -6.58700590e-02],
       [ 7.87481552e-02,  4.44687665e-01],
       [-3.20283657e-01, -4.91152823e-01],
       [-1.79683001e-01, -3.67135590e-01],
       [-1.25881087e-01,  2.83354965e-02],
       [ 2.54756964e-01,  4.76272885e-01],
       [ 5.75089732e-02,  3.50456563e-01],
       [-1.10642823e-01, -2.21017756e-01],
       [-2.19219599e-01, -4.54790149e-01],
       [-1.05527997e-01, -1.37505854e-01],
       [-1.53697064e-01, -5.85325909e-01],
       [ 1.93365839e-02,  2.14242949e-01],
       [ 1.63102855e-01,  4.84229724e-01],
       [ 6.98433334e-02,  4.31182350e-01],
       [-2.48572612e-01, -4.73525191e-01],
       [ 1.24705368e-01,  1.89770304e-01],
       [-1.63715904e-02,  1.44990584e-02],
       [-2.52042832e-01, -3.17835658e-01],
       [-8.97624128e-02, -3.25705387e-01],
       [ 1.10602910e-01,  4.92739602e-01],
...
       [-1.34163248e-01, -9.96738904e-02],
       [-7.16345

In [6]:
# проектируем новые данные для Supervised PCA
W_mean_spca = trace_spca.posterior["W"].mean(("chain", "draw")).to_numpy()  # получаем среднее W

# генерируем новые данные X_new (например, 10 новых наблюдений)
X_new_spca = np.random.randn(10, W_mean_spca.shape[0])

# вычисляем Z_new (PCA scores для новых данных)
W_cov_inv_spca = np.linalg.inv(W_mean_spca.T @ W_mean_spca)  # (k x k), инвертируем ковариационную матрицу
Z_new_spca = X_new_spca @ W_mean_spca @ W_cov_inv_spca  # Shape (10, k)
print("Supervised PCA - Projected Z:", Z_new_spca)

Supervised PCA - Projected Z: [[-5.10237969  0.76022198]
 [-1.81814873 -1.38118457]
 [ 6.35274723 -0.20988809]
 [-7.1058859   2.35408292]
 [-2.14493512 -1.29980715]
 [ 3.23227509 -4.49923301]
 [ 2.65668817 -2.38703935]
 [-3.13059859  4.43806132]
 [ 0.47441167 -1.76636804]
 [-1.83776061  4.39477322]]


# PLS

In [7]:
# генерируем синтетические данные для PLS
np.random.seed(42)
n_samples, n_features, latent_dim = 100, 5, 2
true_W_pls = np.random.randn(n_features, latent_dim)
true_Z_pls = np.random.randn(n_samples, latent_dim)
true_y_pls = true_Z_pls @ np.random.randn(latent_dim, 1) + np.random.normal(0, 0.1, (n_samples, 1))  # целевая переменная
X_pls = true_Z_pls @ true_W_pls.T + np.random.normal(0, 0.1, (n_samples, n_features))  # наблюдаемые данные

# PyMC Model for Partial Least Squares (PLS)
with pm.Model() as pls_model:
    # Priors
    W_pls = pm.Normal("W", mu=0, sigma=1, shape=(n_features, latent_dim))  # матрица весов
    Z_pls = pm.Normal("Z", mu=0, sigma=1, shape=(n_samples, latent_dim))  # скрытые переменные
    sigma_pls = pm.HalfNormal("sigma", sigma=1)  # стандартное отклонение шума
    beta_pls = pm.Normal("beta", mu=0, sigma=1, shape=(latent_dim, 1))  # кфы для целевых переменных
    mu_y_pls = pm.Normal("mu_y", mu=0, sigma=1)  # смещение для целевых переменных

    # ликвидность
    X_obs_pls = pm.Normal("X_obs", mu=pm.math.dot(Z_pls, W_pls.T), sigma=sigma_pls, observed=X_pls)
    y_obs_pls = pm.Normal("y_obs", mu=pm.math.dot(Z_pls, beta_pls) + mu_y_pls, sigma=sigma_pls, observed=true_y_pls)

    # инференс
    trace_pls = pm.sample(1000, return_inferencedata=True)


# извлечение постериорных суммарных данных
summary_pls = az.summary(trace_pls)
print(summary_pls)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [W, Z, sigma, beta, mu_y]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 26 seconds.
There were 1 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


             mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
W[0, 0]     0.101  0.266  -0.354    0.486      0.105    0.078       7.0   
W[0, 1]    -0.318  0.186  -0.535    0.123      0.044    0.032      17.0   
W[1, 0]    -0.671  1.228  -1.864    1.699      0.537    0.403       6.0   
W[1, 1]     0.137  1.057  -1.688    1.697      0.429    0.319       7.0   
W[2, 0]     0.098  0.261  -0.362    0.373      0.117    0.089       6.0   
...           ...    ...     ...      ...        ...      ...       ...   
Z[99, 1]    0.800  2.199  -3.394    3.767      0.912    0.681       6.0   
beta[0, 0] -0.501  0.960  -1.440    1.322      0.423    0.318       6.0   
beta[1, 0]  0.059  0.815  -1.320    1.294      0.329    0.245       7.0   
mu_y       -0.002  0.012  -0.025    0.019      0.000    0.000    3345.0   
sigma       0.100  0.004   0.093    0.107      0.000    0.000    1711.0   

            ess_tail  r_hat  
W[0, 0]         24.0   1.53  
W[0, 1]         59.0   1.20  
W[1, 0]  

In [8]:
# получаем скрытые переменные (PCA Scores)
Z_mean_pls = trace_pls.posterior["Z"].mean(("chain", "draw"))
print("PLS Scores (Z):", Z_mean_pls)

PLS Scores (Z): <xarray.DataArray 'Z' (Z_dim_0: 100, Z_dim_1: 2)>
array([[ 0.11368726,  0.22538157],
       [ 0.77084308, -0.7016099 ],
       [-0.0163133 ,  1.20854003],
       [-0.22511765,  0.79132643],
       [ 0.50936592,  0.19352608],
       [ 0.21939069, -1.06580368],
       [ 0.62286941, -0.54591758],
       [-0.12220157,  0.47961189],
       [-0.29233559,  0.97532673],
       [ 0.04993754,  0.36722882],
       [-0.82971386,  1.02120058],
       [ 0.4325448 , -0.30166422],
       [ 0.55732864, -0.90685023],
       [ 0.76540327, -0.67217685],
       [-0.229486  ,  1.05747003],
       [ 0.00291643, -0.46674412],
       [ 0.0993135 ,  0.01530416],
       [ 0.14328781,  0.86806967],
       [-0.4417937 ,  0.57523787],
       [ 0.76303771, -0.8212977 ],
...
       [ 0.24133346,  0.37258768],
       [-0.11771253,  0.08347472],
       [-0.33692861,  0.10309403],
       [-0.54939853,  0.35261369],
       [-1.09996178,  0.95345262],
       [ 0.39063004, -0.66135439],
       [-0.32881265,

In [9]:
# проектируем новые данные для Partial Least Squares (PLS)
W_mean_pls = trace_pls.posterior["W"].mean(("chain", "draw")).to_numpy()  # получаем среднее W

# генерируем новые данные X_new для PLS
X_new_pls = np.random.randn(10, W_mean_pls.shape[0])

# вычисляем Z_new (PLS scores для новых данных)
W_cov_inv_pls = np.linalg.inv(W_mean_pls.T @ W_mean_pls)  # (k x k), инвертируем ковариационную матрицу
Z_new_pls = X_new_pls @ W_mean_pls @ W_cov_inv_pls  # Shape (10, k)
print("PLS - Projected Z:", Z_new_pls)

PLS - Projected Z: [[ 0.82934694  1.05157958]
 [-0.07171634  1.04111637]
 [-0.87001138 -1.60935822]
 [ 1.44472947  0.94398891]
 [-0.00927304  1.09720955]
 [-1.42164735  0.95441025]
 [-0.8751863   0.25594804]
 [ 1.3924251  -0.95812258]
 [-0.45549468  0.58612098]
 [ 1.21762418 -1.28369111]]


# CCA

In [10]:
# генерируем синтетические данные для CCA
np.random.seed(42)
n_samples, n_features_x, n_features_y, latent_dim = 100, 5, 4, 2
true_W_x = np.random.randn(n_features_x, latent_dim)
true_W_y = np.random.randn(n_features_y, latent_dim)
true_Z_cc = np.random.randn(n_samples, latent_dim)
true_X = true_Z_cc @ true_W_x.T + np.random.normal(0, 0.1, (n_samples, n_features_x))  # наблюдаемые данные X
true_Y = true_Z_cc @ true_W_y.T + np.random.normal(0, 0.1, (n_samples, n_features_y))  # наблюдаемые данные Y

# PyMC Model for Canonical Correlation Analysis (CCA)
with pm.Model() as cca_model:
    W_x = pm.Normal("W_x", mu=0, sigma=1, shape=(n_features_x, latent_dim))  # матрица весов для X
    W_y = pm.Normal("W_y", mu=0, sigma=1, shape=(n_features_y, latent_dim))  # матрица весов для Y
    Z_cc = pm.Normal("Z_cc", mu=0, sigma=1, shape=(n_samples, latent_dim))  # скрытые переменные
    sigma_cc = pm.HalfNormal("sigma", sigma=1)  # стандартное отклонение шума

    X_obs_cc = pm.Normal("X_obs", mu=pm.math.dot(Z_cc, W_x.T), sigma=sigma_cc, observed=true_X)
    Y_obs_cc = pm.Normal("Y_obs", mu=pm.math.dot(Z_cc, W_y.T), sigma=sigma_cc, observed=true_Y)

    # инференс
    trace_cca = pm.sample(1000, return_inferencedata=True)

# извлечение постериорных суммарных данных для CCA
summary_cca = az.summary(trace_cca)
print(summary_cca)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [W_x, W_y, Z_cc, sigma]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 28 seconds.
There were 27 divergences after tuning. Increase `target_accept` or reparameterize.
Chain 3 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


              mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
W_x[0, 0]   -0.084  0.334  -0.480    0.477      0.145    0.109       6.0   
W_x[0, 1]   -0.185  0.240  -0.531    0.226      0.045    0.045      28.0   
W_x[1, 0]   -0.112  1.144  -1.778    1.730      0.389    0.285      10.0   
W_x[1, 1]    0.257  1.286  -1.791    1.817      0.562    0.422       6.0   
W_x[2, 0]    0.042  0.234  -0.345    0.360      0.085    0.063       8.0   
...            ...    ...     ...      ...        ...      ...       ...   
Z_cc[98, 0]  0.089  0.595  -0.837    0.920      0.230    0.171       7.0   
Z_cc[98, 1]  0.348  0.517  -0.665    0.971      0.184    0.135      10.0   
Z_cc[99, 0]  0.184  0.701  -0.956    1.042      0.296    0.221       6.0   
Z_cc[99, 1]  0.253  0.601  -0.831    1.069      0.196    0.143      11.0   
sigma        0.099  0.003   0.094    0.104      0.000    0.000    1633.0   

             ess_tail  r_hat  
W_x[0, 0]        46.0   1.68  
W_x[0, 1]        23.0   1

In [11]:
# получаем скрытые переменные (CCA Scores)
Z_mean_cca = trace_cca.posterior["Z_cc"].mean(("chain", "draw"))
print("CCA Scores (Z):", Z_mean_cca)

CCA Scores (Z): <xarray.DataArray 'Z_cc' (Z_cc_dim_0: 100, Z_cc_dim_1: 2)>
array([[ 1.95760592e-01, -1.37428973e-02],
       [-3.38239233e-01, -6.45917011e-01],
       [-3.87098244e-02, -4.17973662e-01],
       [ 1.33920108e-01,  2.69427054e-01],
       [ 2.72178817e-01,  5.65331020e-01],
       [ 1.16860982e-01,  1.08042348e-01],
       [ 1.54812834e-01,  7.29402703e-01],
       [-2.52785943e-02, -3.07568247e-01],
       [-2.06751708e-01, -6.50495778e-01],
       [-8.38379272e-02, -6.49679009e-01],
       [ 3.09470373e-01,  5.78999215e-01],
       [-1.61707131e-01, -2.47034097e-01],
       [ 2.14717931e-02, -4.26267351e-02],
       [ 3.38775801e-01,  4.25132000e-01],
       [ 1.19642004e-01,  4.70724990e-01],
       [-1.09340155e-01, -6.28861857e-01],
       [-7.14653751e-02, -2.15832782e-01],
       [ 1.45573505e-01,  3.93939003e-01],
       [-2.30341030e-01, -1.69327886e-01],
       [ 1.78954163e-01,  2.33715323e-01],
...
       [ 9.59394051e-02,  8.36334736e-01],
       [-1.4841858

In [12]:
# проектируем новые данные для Canonical Correlation Analysis (CCA)
W_mean_x = trace_cca.posterior["W_x"].mean(("chain", "draw")).to_numpy()  # получаем среднее W для X
W_mean_y = trace_cca.posterior["W_y"].mean(("chain", "draw")).to_numpy()  # получаем среднее W для Y

# генерируем новые данные X_new и Y_new для CCA
X_new_cca = np.random.randn(10, W_mean_x.shape[0])  # Shape (10, n_features_x)
Y_new_cca = np.random.randn(10, W_mean_y.shape[0])  # Shape (10, n_features_y)

# вычисляем Z_new (CCA scores для новых данных)
W_cov_inv_x = np.linalg.inv(W_mean_x.T @ W_mean_x)  # (k x k), инвертируем ковариационную матрицу для X
Z_new_x_cca = X_new_cca @ W_mean_x @ W_cov_inv_x  # Shape (10, k)

W_cov_inv_y = np.linalg.inv(W_mean_y.T @ W_mean_y)  # (k x k), инвертируем ковариационную матрицу для Y
Z_new_y_cca = Y_new_cca @ W_mean_y @ W_cov_inv_y  # Shape (10, k)

print("CCA - Projected Z for X:", Z_new_x_cca)
print("CCA - Projected Z for Y:", Z_new_y_cca)

CCA - Projected Z for X: [[-5.12028937  2.15996226]
 [-4.85207028  2.7918088 ]
 [-1.80562497  2.5605011 ]
 [-2.02258471 -2.64786877]
 [-5.61500148  0.48032508]
 [ 3.3076212  -2.04763165]
 [ 6.47475177 -2.81812604]
 [-1.31754247  6.57423623]
 [-6.73401847 -2.94887844]
 [-4.11012743  2.95305918]]
CCA - Projected Z for Y: [[ 4.89247451 -1.66023165]
 [-2.45019444  0.20693743]
 [-6.00246065  0.65852092]
 [ 4.30781041 -3.01005908]
 [-2.24164103 -0.89352952]
 [-2.27543163  0.84682058]
 [-4.24310646  0.34641681]
 [ 3.00520583 -0.47213467]
 [ 0.10615488  0.76125885]
 [-2.49751198 -0.0752903 ]]


# Дополнительно:
попробуйте сравнить для нескольких датасетов, как работает PCA и Supervised PCA для уменьшения размерности и последующего обучения supervised моделей.

In [13]:
import pymc as pm
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import arviz as az

Сгенерируем данные для датасетов

In [14]:
n_samples, n_features, latent_dim = 100, 5, 2

# генерируем 3 разных набора данных
datasets = []
for _ in range(3):
    true_W = np.random.randn(n_features, latent_dim)
    true_Z = np.random.randn(n_samples, latent_dim)
    true_y = true_Z @ np.random.randn(latent_dim, 1) + np.random.normal(0, 0.1, (n_samples, 1))  # целевая переменная
    X = true_Z @ true_W.T + np.random.normal(0, 0.1, (n_samples, n_features))  # наблюдаемые данные
    y = (true_y > 0).astype(int)  # бинарная целевая переменная (классификация)
    datasets.append((X, y))

In [19]:
def evaluate_model(X_train, X_test, y_train, y_test):
    logreg = LogisticRegression()

    with pm.Model() as pca_model:
        W = pm.Normal("W", mu=0, sigma=1, shape=(n_features, latent_dim))  # Weight matrix
        Z = pm.Normal("Z", mu=0, sigma=1, shape=(n_samples, latent_dim))  # Latent variables
        sigma = pm.HalfNormal("sigma", sigma=1)  # Noise standard deviation
        X_obs = pm.Normal("X_obs", mu=pm.math.dot(Z, W.T), sigma=sigma, observed=X)

        trace_pca = pm.sample(500, return_inferencedata=True)

    Z_mean_pca = trace_pca.posterior["Z"].mean(("chain", "draw"))

    # преобразуем данные для обучения
    X_train_pca = Z_mean_pca[:len(X_train), :]
    X_test_pca = Z_mean_pca[len(X_train):, :]

    # обучаем модель на сниженной размерности
    logreg.fit(X_train_pca, y_train)
    y_pred_pca = logreg.predict(X_test_pca)

    # Supervised PCA
    with pm.Model() as spca_model:
        W = pm.Normal("W", mu=0, sigma=1, shape=(n_features, latent_dim))
        Z = pm.Normal("Z", mu=0, sigma=1, shape=(n_samples, latent_dim))
        sigma = pm.HalfNormal("sigma", sigma=1)
        beta = pm.Normal("beta", mu=0, sigma=1, shape=(latent_dim, 1))
        mu_y = pm.Normal("mu_y", mu=0, sigma=1)

        X_obs = pm.Normal("X_obs", mu=pm.math.dot(Z, W.T), sigma=sigma, observed=X)
        y_obs = pm.Normal("y_obs", mu=pm.math.dot(Z, beta) + mu_y, sigma=sigma, observed=true_y)

        trace_spca = pm.sample(500, return_inferencedata=True)

    Z_mean_spca = trace_spca.posterior["Z"].mean(("chain", "draw"))

    # преобразуем данные для обучения
    X_train_spca = Z_mean_spca[:len(X_train), :]
    X_test_spca = Z_mean_spca[len(X_train):, :]

    # обучаем модель на сниженной размерности
    logreg.fit(X_train_spca, y_train)
    y_pred_spca = logreg.predict(X_test_spca)

    accuracy_pca = accuracy_score(y_test, y_pred_pca)
    accuracy_spca = accuracy_score(y_test, y_pred_spca)

    return accuracy_pca, accuracy_spca

In [21]:
results = []
for i, (X, y) in enumerate(datasets):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
    accuracy_pca, accuracy_spca = evaluate_model(X_train, X_test, y_train, y_test)
    results.append({
        'Dataset': f'Dataset {i+1}',
        'Bayesian PCA Accuracy': accuracy_pca,
        'Supervised PCA Accuracy': accuracy_spca
    })

results_df = pd.DataFrame(results)
print(results_df)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [W, Z, sigma]


Output()

Sampling 4 chains for 1_000 tune and 500 draw iterations (4_000 + 2_000 draws total) took 12 seconds.
There were 500 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
/Users/elizabeth/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [W, Z, sigma, beta, mu_y]


Output()

Sampling 4 chains for 1_000 tune and 500 draw iterations (4_000 + 2_000 draws total) took 5 seconds.
There were 9 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
/Users/elizabeth/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [W, Z, sigma]


...

Sampling 4 chains for 1_000 tune and 500 draw iterations (4_000 + 2_000 draws total) took 16 seconds.
There were 522 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
/Users/elizabeth/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [W, Z, sigma, beta, mu_y]


Output()

Sampling 4 chains for 1_000 tune and 500 draw iterations (4_000 + 2_000 draws total) took 26 seconds.
There were 9 divergences after tuning. Increase `target_accept` or reparameterize.
Chain 1 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


     Dataset  Bayesian PCA Accuracy  Supervised PCA Accuracy
0  Dataset 1               0.366667                 0.366667
1  Dataset 2               0.500000                 0.533333
2  Dataset 3               0.533333                 0.566667


/Users/elizabeth/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Видно, что в целом supervised pca работает также, как pca, либо лучше. На 1 датасете точность очень низкая, это может быть связано с тем, что в данных есть слишком сильный шум, на 2 и 3ем датасетах результаты лучше, вероятно данные имеют более четкие линейные зависимости друг с другом